<a href="https://colab.research.google.com/github/kalyankalluri999/Academic-Projects/blob/main/Copy_of_Testing_PHASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# import the libraries as shown below
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [4]:
import os
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from IPython.display import SVG
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.utils import to_categorical, model_to_dot, plot_model

In [5]:
train_path = ('/content/drive/MyDrive/Newfolder/TEST_data')
valid_path =    ('/content/drive/MyDrive/Newfolder/TRAIN_data')
IMAGE_SIZE = [224, 224]

In [5]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [6]:
folders = glob('/content/drive/MyDrive/Newfolder/TEST_data/*')

In [7]:
# our layers - you can add more if you want

from keras import models
from keras import layers
from keras.layers import GlobalAveragePooling2D, Dense, Flatten, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import plot_model

model = models.Sequential()
model.add(inception) # add pre_trained layers
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.4))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
x = Flatten()(inception.output)

NameError: ignored

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import optimizers
model.compile(Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
#Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Newfolder/TEST_data',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Newfolder/TRAIN_data',
                                            target_size = (224, 224),
                                            batch_size = 64,
                                            class_mode = 'categorical')

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
import matplotlib.pyplot as plt
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# save it as a h5 file
from tensorflow.keras.models import load_model
model.save('model_inception.h5')

In [ ]:
y_pred = model.predict(test_set)
y_pred

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

In [ ]:
import cv2
import tensorflow as tf
def prepare(filepath):
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img_array = img_array / 255
    new_array = cv2.resize(img_array, (224, 224))
    return new_array.reshape(-1, 224, 224, 3)

model = tf.keras.models.load_model('model_inception.h5')

In [ ]:
class_dict = training_set.class_indices
class_dict

In [ ]:
def prediction_cls(prediction):
    for key, clss in class_dict.items():
        if np.argmax(prediction) == clss:
            return key

In [ ]:
prediction = model.predict([prepare('/content/drive/MyDrive/Newfolder/TEST_data/Chow/n02112137_1005-0.jpg')])
prediction_cls(prediction)

In [ ]:
prediction = model.predict([prepare('/content/drive/MyDrive/Newfolder/TEST_data/Cairn/n02096177_1000-0.jpg')])
prediction_cls(prediction)

In [ ]:
from PIL import Image
imageno=np.random.random_integers(low=0, high=test_set.samples)

name = test_set.filepaths[imageno]
print(name)
plt.imshow(mpimg.imread(name))

img = Image.open(test_set.filepaths[imageno]).resize((IMAGE_SIZE))
probabilities = model.predict(preprocess_input(np.expand_dims(img, axis=0)))
breed_list = tuple(zip(test_set.class_indices.values(), test_set.class_indices.keys()))

for i in probabilities[0].argsort()[-5:][::-1]: 
    print(probabilities[0][i], "  :  " , breed_list[i])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
test_set.reset()
predictions = model.predict_generator(test_set, steps=len(test_set))
y = np.argmax(predictions, axis=1)
print('Classification Report')
cr = classification_report(y_true=test_set.classes, y_pred=y, target_names=test_set.class_indices)
print(cr)